In [1]:
import pandas as pd
import pymongo

In [2]:
df = pd.read_csv("../Ryan_Cleaned_Data/CombinedTotal.csv", low_memory = False)

In [3]:
# Combine region_1 and region_2 into a single column using lists
region_list = []

for thing in df.iterrows():
    region_1 = str(list(thing)[1].loc["Region_1"])
    region_2 = str(list(thing)[1].loc["Region_2"])

# list containing the number of region entries for each row
    if region_2 == "nan":
        if region_1 == "nan":
            region = []
        else:
            region = [region_1]
    else:
        region = [region_1, region_2]

    region_list.append(region)

df["Region"] = region_list
    
df.head()

,Country,Description,Designation,Points,Price,Province,Region_1,Region_2,Variety,Winery,Title,Region
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,"[Napa Valley, Napa]"
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,[Toro]
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,"[Knights Valley, Sonoma]"
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,"[Willamette Valley, Willamette Valley]"
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,[Bandol]


In [4]:
# build dataframe of info for each winery
winery_df = pd.DataFrame()

In [5]:
# mean points and prices
winery_df[["Points_Mean", "Price_Mean"]] = df[["Winery", "Points", "Price"]].groupby("Winery").mean().round(1)

In [6]:
# median points and prices
winery_df[["Points_Median", "Price_Median"]] = df[["Winery", "Points", "Price"]].groupby("Winery").median()

In [7]:
# total number of reviews
winery_df["Total_Reviews"] = df[["Winery", "Description"]].groupby("Winery").count()

In [8]:
# count the number of different wine varieties
winery_df["N_Varieties"] = df[["Winery", "Variety"]].groupby("Winery").nunique()

In [9]:
# list of each type of wine variety
wine_types = df[["Winery", "Variety"]].groupby("Winery")["Variety"].unique()
wine_list = []

for item in wine_types:
    wines = item.tolist()
    wine_list.append(wines)
    
winery_df["Varieties"] = wine_list

In [10]:
# geographical data
winery_df[["Country", "Province", "Region"]] = df[["Winery", "Country", "Province", "Region"]].groupby("Winery").first()

In [11]:
# add winery name as a column instead of an index
winery_df = winery_df.reset_index()

In [12]:
winery_df.sort_values("Points_Mean", ascending = False)

,Winery,Points_Mean,Price_Mean,Points_Median,Price_Median,Total_Reviews,N_Varieties,Varieties,Country,Province,Region
16194,Sloan,100.0,245.0,100.0,245.0,1,1,[Cabernet Blend],US,California,"[Rutherford, Napa]"
754,Araujo,97.0,230.0,97.0,230.0,2,1,[Cabernet Sauvignon],US,California,"[Napa Valley, Napa]"
8903,Gandona,97.0,190.0,97.0,190.0,1,1,[Cabernet Sauvignon],US,California,"[Napa Valley, Napa]"
13555,Ovid,97.0,195.0,97.0,195.0,1,1,[Bordeaux-style Red Blend],US,California,"[Napa Valley, Napa]"
4487,Château Pétrus,96.4,2000.0,96.0,2000.0,11,1,[Bordeaux-style Red Blend],France,Bordeaux,[Pomerol]
...,...,...,...,...,...,...,...,...,...,...,...
7843,Eleusis,80.0,19.0,80.0,19.0,1,1,[Albariño],Spain,Galicia,[Rías Baixas]
1792,Bodegas del Mundo,80.0,8.0,80.0,8.0,3,3,"[Tempranillo, Garnacha, Viura]",Spain,Northern Spain,[Rioja]
17483,TorresManur,80.0,9.0,80.0,9.0,1,1,[Verdejo-Viura],Spain,Northern Spain,[Rueda]
7809,El Morrión,80.0,9.0,80.0,9.0,1,1,[Tempranillo],Spain,Central Spain,[Vino de la Tierra de Castilla]


In [13]:
winery_df.to_csv("wineries.csv")

In [14]:
# create mongo database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.winery_db

In [15]:
# convert winery dataframe to dictionary
winery_dict = winery_df.fillna("").to_dict(orient = "index")

In [16]:
# insert dictionary into winery_db.data
db.data.insert_many(winery_dict.values())